### 作業
目前已經學過許多的模型，相信大家對整體流程應該比較掌握了，這次作業請改用**手寫辨識資料集**，步驟流程都是一樣的，請試著自己撰寫程式碼來完成所有步驟

In [1]:
# 將需要的都import進來
import os
import copy
import time
import math
import numpy             as np
import pandas            as pd
import seaborn           as sns
import datetime          as dt
import warnings
import matplotlib.pyplot as plt

from sklearn.linear_model    import LogisticRegression,LinearRegression,Lasso,Ridge
from sklearn.tree            import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble        import RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.metrics         import roc_curve,mean_squared_error,r2_score,accuracy_score,precision_score,recall_score,fbeta_score
from sklearn.datasets        import load_boston,load_wine,load_digits
from sklearn.preprocessing   import LabelEncoder,MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.model_selection import cross_val_score,train_test_split

from scipy                   import stats
from itertools               import compress
from IPython.display         import YouTubeVideo

# 將較長的函式改名一下
MSE  = mean_squared_error
ACC  = accuracy_score
MME  = MinMaxScaler()
LE   = LabelEncoder()
OHE  = OneHotEncoder()

# 一些必要的設定
warnings.filterwarnings('ignore')
%matplotlib inline

# 設定【data的資料夾路徑】，命名為【data_folder】
data_folder = 'C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data'

In [2]:
# 讀取 digits 資料
t001 = load_digits()
# 轉成dataframe觀看資料：X
train_X_t1 = pd.DataFrame(t001.data)
print(train_X_t1.shape)
print(train_X_t1.head())
# 轉成dataframe觀看資料：Y
train_Y_t1 = pd.DataFrame({"target": t001.target})
print(train_Y_t1.shape)
print(train_Y_t1.head())

(1797, 64)
    0    1    2     3     4     5    6    7    8    9   ...   54   55   56  \
0  0.0  0.0  5.0  13.0   9.0   1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1  0.0  0.0  0.0  12.0  13.0   5.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2  0.0  0.0  0.0   4.0  15.0  12.0  0.0  0.0  0.0  0.0  ...  5.0  0.0  0.0   
3  0.0  0.0  7.0  15.0  13.0   1.0  0.0  0.0  0.0  8.0  ...  9.0  0.0  0.0   
4  0.0  0.0  0.0   1.0  11.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

    57   58    59    60    61   62   63  
0  0.0  6.0  13.0  10.0   0.0  0.0  0.0  
1  0.0  0.0  11.0  16.0  10.0  0.0  0.0  
2  0.0  0.0   3.0  11.0  16.0  9.0  0.0  
3  0.0  7.0  13.0  13.0   9.0  0.0  0.0  
4  0.0  0.0   2.0  16.0   4.0  0.0  0.0  

[5 rows x 64 columns]
(1797, 1)
   target
0       0
1       1
2       2
3       3
4       4


In [3]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train_X_t1, train_Y_t1, test_size=0.25, random_state=4)
# 看切完長怎樣
print(x_train.shape)
print(x_train.head())
print(y_train.shape)
print(y_train.head())
print(x_test.shape)
print(x_test.head())
print(y_test.shape)
print(y_test.head())

(1347, 64)
       0    1    2     3     4     5    6    7    8    9   ...    54   55  \
309   0.0  0.0  4.0  13.0  16.0  14.0  2.0  0.0  0.0  2.0  ...   8.0  0.0   
550   0.0  0.0  2.0  13.0  16.0   8.0  0.0  0.0  0.0  0.0  ...   5.0  0.0   
1115  0.0  0.0  1.0  13.0   2.0   0.0  0.0  0.0  0.0  0.0  ...  14.0  5.0   
1227  0.0  0.0  6.0  16.0  11.0   0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0   
1732  0.0  0.0  4.0  16.0   6.0   0.0  0.0  0.0  0.0  0.0  ...   5.0  0.0   

       56   57   58    59    60    61    62   63  
309   0.0  0.0  3.0  15.0  16.0  11.0   1.0  0.0  
550   0.0  0.0  3.0  15.0  16.0  16.0   3.0  0.0  
1115  0.0  0.0  1.0  11.0  16.0  16.0  13.0  1.0  
1227  0.0  0.0  4.0  15.0  16.0   3.0   0.0  0.0  
1732  0.0  0.0  5.0  15.0  14.0   7.0   0.0  0.0  

[5 rows x 64 columns]
(1347, 1)
      target
309        8
550        6
1115       6
1227       1
1732       6
(450, 64)
       0    1     2     3     4     5    6    7    8    9   ...   54   55  \
492   0.0  0.0   0.0  

In [4]:
help(GradientBoostingClassifier())

Help on GradientBoostingClassifier in module sklearn.ensemble.gradient_boosting object:

class GradientBoostingClassifier(BaseGradientBoosting, sklearn.base.ClassifierMixin)
 |  GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto', validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)
 |  
 |  Gradient Boosting for classification.
 |  
 |  GB builds an additive model in a
 |  forward stage-wise fashion; it allows for the optimization of
 |  arbitrary differentiable loss functions. In each stage ``n_classes_``
 |  regression trees are fit on the negative gradient of the
 |  binomial or multinomial deviance loss function. Binary classification
 |  is a special case whe

In [5]:
# GBC1：使用預設值
GBC1 = GradientBoostingClassifier(loss='deviance',
                                  learning_rate=0.1,
                                  n_estimators=100,
                                  subsample=1.0,
                                  criterion='friedman_mse',
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  min_weight_fraction_leaf=0.0,
                                  max_depth=3,
                                  min_impurity_decrease=0.0,
                                  min_impurity_split=None,
                                  init=None,
                                  random_state=None,
                                  max_features=None,
                                  verbose=0,
                                  max_leaf_nodes=None,
                                  warm_start=False,
                                  presort='auto',
                                  validation_fraction=0.1,
                                  n_iter_no_change=None,
                                  tol=0.0001)
  
GBC1.fit(x_train, y_train)
print(GBC1.score(x_train, y_train))
# 將x_test丟進上面跑好的回歸模型裡，得到y_pred，也就是預測出來的y_pred。
y_pred = GBC1.predict(x_test)
print(y_pred.shape)
print(pd.DataFrame(y_pred).head())
# 看一下預測出來的y_pred和實際的y_test差多少？
print("Mean squared error: %.2f"% MSE(y_test, y_pred))
print("Accuracy: ", ACC(y_test, y_pred))

1.0
(450,)
   0
0  8
1  7
2  0
3  5
4  3
Mean squared error: 0.50
Accuracy:  0.9644444444444444


In [6]:
# RFC1：使用預設值
# 結果：100%正確
RFC1 = RandomForestClassifier(n_estimators='warn',
                              criterion='gini',
                              max_depth=None,
                              min_samples_split=2,
                              min_samples_leaf=1,
                              min_weight_fraction_leaf=0.0,
                              max_features='auto',
                              max_leaf_nodes=None,
                              min_impurity_decrease=0.0,
                              min_impurity_split=None,
                              bootstrap=True, oob_score=False,
                              n_jobs=None, random_state=None,
                              verbose=0,
                              warm_start=False,
                              class_weight=None)
RFC1.fit(x_train, y_train)
print(RFC1.score(x_train, y_train))
# 將x_test丟進上面跑好的回歸模型裡，得到y_pred，也就是預測出來的y_pred。
y_pred = RFC1.predict(x_test)
print(y_pred.shape)
print(pd.DataFrame(y_pred).head())
# 看一下預測出來的y_pred和實際的y_test差多少？
print("Mean squared error: %.2f"% MSE(y_test, y_pred))
print("Accuracy: ", ACC(y_test, y_pred))

0.9992576095025983
(450,)
   0
0  1
1  7
2  0
3  5
4  3
Mean squared error: 1.33
Accuracy:  0.9533333333333334


In [7]:
# LGR
# 跑完背後就已經有整個回歸模型了
LGR = LogisticRegression()
LGR.fit(x_train, y_train)
# 印出coef
print(LGR.score(x_train, y_train))
# 將x_test丟進上面跑好的回歸模型裡，得到y_pred，也就是預測出來的y_pred。
y_pred = LGR.predict(x_test)
print(y_pred.shape)
print(pd.DataFrame(y_pred).head())
# 看一下預測出來的y_pred和實際的y_test差多少？
print("Mean squared error: %.2f"% MSE(y_test, y_pred))
# 本例中直接分成了0,1,2，屬於classification tasks，可以使用ACC。
print("Accuracy: ", ACC(y_test, y_pred))

0.9962880475129918
(450,)
   0
0  8
1  7
2  0
3  5
4  3
Mean squared error: 0.60
Accuracy:  0.9533333333333334


## 集中於Day_042_HW